# Review Analysis

In this exercise, you will go back to the Olist dataset. Run the code below to load the data.

In [2]:
import pandas as pd

df = pd.read_csv("https://wagon-public-datasets.s3.amazonaws.com/Machine%20Learning%20Datasets/reviews.csv")

df.head()

/Users/selmalopez/.pyenv/versions/3.8.12/envs/lewagon_current/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (1,2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,review_id,length_review,review_score,order_id,product_category_name,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,7bc2406110b926393aa56f80a40eba40,0,4,73fc7af87114b39712e6da79b0a377eb,esporte_lazer,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59,41dcb106f807e993532d446263290104,delivered,2018-01-11 15:30:49,2018-01-11 15:47:59,2018-01-12 21:57:22,2018-01-17 18:42:41,2018-02-02 00:00:00
1,80e641a11e56f04c1ad469d5645fdfde,0,5,a548910a1c6147796b98fdf73dbeba33,informatica_acessorios,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13,8a2e7ef9053dea531e4dc76bd6d853e6,delivered,2018-02-28 12:25:19,2018-02-28 12:48:39,2018-03-02 19:08:15,2018-03-09 23:17:20,2018-03-14 00:00:00
2,228ce5500dc1d8e020d8d1322874b6f0,0,5,f9e4b658b201a9f2ecdecbb34bed034b,informatica_acessorios,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24,e226dfed6544df5b7b87a48208690feb,delivered,2018-02-03 09:56:22,2018-02-03 10:33:41,2018-02-06 16:18:28,2018-02-16 17:28:48,2018-03-09 00:00:00
3,e64fb393e7b32834bb789ff8bb30750e,37,5,658677c97b385a9be170737859d3511b,ferramentas_jardim,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06,de6dff97e5f1ba84a3cd9a3bc97df5f6,delivered,2017-04-09 17:41:13,2017-04-09 17:55:19,2017-04-10 14:24:47,2017-04-20 09:08:35,2017-05-10 00:00:00
4,f7c4243c7fe1938f181bec41a392bdeb,100,5,8e6bfb81e283fa7e4f11123a3fb894f1,esporte_lazer,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53,5986b333ca0d44534a156a52a8e33a83,delivered,2018-02-10 10:59:03,2018-02-10 15:48:21,2018-02-15 19:36:14,2018-02-28 16:33:35,2018-03-09 00:00:00


👇 Analyse the comment reviews to understand what causes the bad review scores.

In [ ]:
#pourquoi les commentaires des reviews sont mauvaises 

In [3]:
#imports
import string 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB

In [4]:
df = df[(df['review_creation_date'] <= df['order_estimated_delivery_date'])]

df.head()
#filter out the dates

,review_id,length_review,review_score,order_id,product_category_name,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,7bc2406110b926393aa56f80a40eba40,0,4,73fc7af87114b39712e6da79b0a377eb,esporte_lazer,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59,41dcb106f807e993532d446263290104,delivered,2018-01-11 15:30:49,2018-01-11 15:47:59,2018-01-12 21:57:22,2018-01-17 18:42:41,2018-02-02 00:00:00
1,80e641a11e56f04c1ad469d5645fdfde,0,5,a548910a1c6147796b98fdf73dbeba33,informatica_acessorios,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13,8a2e7ef9053dea531e4dc76bd6d853e6,delivered,2018-02-28 12:25:19,2018-02-28 12:48:39,2018-03-02 19:08:15,2018-03-09 23:17:20,2018-03-14 00:00:00
2,228ce5500dc1d8e020d8d1322874b6f0,0,5,f9e4b658b201a9f2ecdecbb34bed034b,informatica_acessorios,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24,e226dfed6544df5b7b87a48208690feb,delivered,2018-02-03 09:56:22,2018-02-03 10:33:41,2018-02-06 16:18:28,2018-02-16 17:28:48,2018-03-09 00:00:00
3,e64fb393e7b32834bb789ff8bb30750e,37,5,658677c97b385a9be170737859d3511b,ferramentas_jardim,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06,de6dff97e5f1ba84a3cd9a3bc97df5f6,delivered,2017-04-09 17:41:13,2017-04-09 17:55:19,2017-04-10 14:24:47,2017-04-20 09:08:35,2017-05-10 00:00:00
4,f7c4243c7fe1938f181bec41a392bdeb,100,5,8e6bfb81e283fa7e4f11123a3fb894f1,esporte_lazer,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53,5986b333ca0d44534a156a52a8e33a83,delivered,2018-02-10 10:59:03,2018-02-10 15:48:21,2018-02-15 19:36:14,2018-02-28 16:33:35,2018-03-09 00:00:00


In [5]:
df = df[['order_id','product_category_name','review_comment_title','review_comment_message','review_score']]

df.head()
#selectionner que les colonnes qui sont en rapport avec le review 

,order_id,product_category_name,review_comment_title,review_comment_message,review_score
0,73fc7af87114b39712e6da79b0a377eb,esporte_lazer,NaN,NaN,4
1,a548910a1c6147796b98fdf73dbeba33,informatica_acessorios,NaN,NaN,5
2,f9e4b658b201a9f2ecdecbb34bed034b,informatica_acessorios,NaN,NaN,5
3,658677c97b385a9be170737859d3511b,ferramentas_jardim,NaN,Recebi bem antes do prazo estipulado.,5
4,8e6bfb81e283fa7e4f11123a3fb894f1,esporte_lazer,NaN,Parabéns lojas lannister adorei comprar pela I...,5


In [6]:
# combine review title and review message

df = df.dropna(subset=['review_comment_title','review_comment_message'])

df['title_comment'] = df["review_comment_title"].fillna('') + " " \
            + df['review_comment_message'].fillna('')



df.head()
#fillna method fill NA/NaN values using the specified method.

,order_id,product_category_name,review_comment_title,review_comment_message,review_score,title_comment
9,b9bf720beb4ab3728760088589c62129,eletroportateis,recomendo,aparelho eficiente. no site a marca do aparelh...,4,recomendo aparelho eficiente. no site a marca ...
15,e51478e7e277a83743b6f9991dbfa3fb,informatica_acessorios,Super recomendo,"Vendedor confiável, produto ok e entrega antes...",5,"Super recomendo Vendedor confiável, produto ok..."
22,4fc44d78867142c627497b60a7e0228a,beleza_saude,Ótimo,Loja nota 10,5,Ótimo Loja nota 10
36,37e7875cdce5a9e5b3a692971f370151,esporte_lazer,Muito bom.,Recebi exatamente o que esperava. As demais en...,4,Muito bom. Recebi exatamente o que esperava. A...
38,e029f708df3cc108b3264558771605c6,pet_shop,Bom,"Recomendo ,",5,"Bom Recomendo ,"


In [ ]:
#unicode to remove accents

In [75]:
import string 
def preprocessing(review_comment_message):
    for i in string.punctuation:
        review_comment_message = review_comment_message.replace(i, '')
    review_comment_message = review_comment_message.lower()
    review_comment_message = review_comment_message.replace('\n', '')
    review_comment_message = review_comment_message.replace('\t', '') 
    review_comment_message = review_comment_message.replace('\r', '') 
    stop_words = set(stopwords.words('portuguese')) 
    word_tokens = word_tokenize(review_comment_message) 
    va = ' '.join(w for w in word_tokens if not w in stop_words)
    return va  

new_df['clean_reviews']= new_df['review_comment_message'].apply(preprocessing)
#NaN values are considered as float, that why we have to drop nan

In [76]:
new_df

,review_score,review_comment_message,clean_reviews
3,5,Recebi bem antes do prazo estipulado.,recebi bem antes prazo estipulado
4,5,Parabéns lojas lannister adorei comprar pela I...,parabéns lojas lannister adorei comprar intern...
9,4,aparelho eficiente. no site a marca do aparelh...,aparelho eficiente site marca aparelho impress...
12,4,"Mas um pouco ,travando...pelo valor ta Boa.\r\n",pouco travandopelo valor ta boa
15,5,"Vendedor confiável, produto ok e entrega antes...",vendedor confiável produto ok entrega antes prazo
...,...,...,...
99416,5,Entregou dentro do prazo. O produto chegou em ...,entregou dentro prazo produto chegou condições...
99422,3,"O produto não foi enviado com NF, não existe v...",produto enviado nf existe venda nf certeza fic...
99428,5,"Excelente mochila, entrega super rápida. Super...",excelente mochila entrega super rápida super r...
99430,1,Solicitei a compra de uma capa de retrovisor c...,solicitei compra capa retrovisor celtaprismame...


In [77]:
new_df.dtypes 
#object is generally a string 

review_score              object
review_comment_message    object
clean_reviews             object
dtype: object

In [78]:
new_df['review_score'].unique()
new_df['review_score'] = new_df['review_score'].map({'1':1,
                            '2': 2, 
                           '3':3,
                           '4':4,
                           '5':5,
                            1:1,
                            2:2,
                            3:3,
                            4:4,
                            5:5 }) 


In [79]:
new_df['review_score'].isna().sum()

2

In [80]:
new_df.dropna(subset=['review_score'],inplace=True)
#inplace pour remplacer ma dataframe

In [81]:
#select only the bad reviews 
#astype retransformer mes colonnes en int
new_df['review_score']= new_df['review_score']
new_df[new_df['review_score'] < 3] 

,review_score,review_comment_message,clean_reviews
16,2.0,"GOSTARIA DE SABER O QUE HOUVE, SEMPRE RECEBI E...",gostaria saber sempre recebi compra agora decp...
19,1.0,Péssimo,péssimo
29,1.0,Não gostei ! Comprei gato por lebre,gostei comprei gato lebre
34,1.0,Sempre compro pela Internet e a entrega ocorre...,sempre compro internet entrega ocorre antes pr...
41,1.0,Nada de chegar o meu pedido.,nada chegar pedido
...,...,...,...
99380,1.0,Boa tarde. \r\nNão recebo todos os produtos fa...,boa tarde recebo todos produtos falta bebe con...
99408,2.0,Foto muito diferente principalmente a graninha...,foto diferente principalmente graninha sintéti...
99411,2.0,Tive um problema na entrega em que o correio c...,problema entrega correio colocou site entregue...
99430,1.0,Solicitei a compra de uma capa de retrovisor c...,solicitei compra capa retrovisor celtaprismame...


In [15]:
relogios_presentes = df[df['product_category_name'].isin(['relogios_presentes'])]

relogios_presentes.head()

,order_id,product_category_name,review_comment_title,review_comment_message,review_score,title_comment
61,8a9424899aac432d80d8e580932b5ee9,relogios_presentes,MT lindo,MT lindo,5,MT lindo MT lindo
338,43050a51a257b17d51295e07650cb654,relogios_presentes,Lamenetável...,"Boa Noite\r\n\r\né lamentável, esta loja que t...",1,"Lamenetável... Boa Noite\r\n\r\né lamentável, ..."
566,0bb77e62d791c3e5b009815ac19825be,relogios_presentes,Bom,Muito bom mesmo!,5,Bom Muito bom mesmo!
705,e87c331d21a3fb3085079a9b32f4a56c,relogios_presentes,10,"Gostei muito , o objeto veio o mesmo comprado,...",5,"10 Gostei muito , o objeto veio o mesmo compra..."
756,93aa1607392862dacfb4b54774480f1d,relogios_presentes,Regular,Loja em si é boa pena a americana nao da supor...,5,Regular Loja em si é boa pena a americana nao...


In [16]:
relogios_presentes_bad_reviews = relogios_presentes[relogios_presentes['review_score'].isin([1])]

relogios_presentes_bad_reviews.head()

,order_id,product_category_name,review_comment_title,review_comment_message,review_score,title_comment
66305,6424ddd05ffee73de1df9f8f9d1bfc25,relogios_presentes,Produto defeito,Não está funcionando o produto,1,Produto defeito Não está funcionando o produto
66836,13fe5b8b3ad46c558f0c53939589776b,relogios_presentes,Ruim,O relógio mau teve contato com a água e já parou,1,Ruim O relógio mau teve contato com a água e j...
67782,8c6541ccd245c735762e6dfbeeeef23e,relogios_presentes,Produto site é um,"Produto anunciado era um e chegou outro, na ve...",1,Produto site é um Produto anunciado era um e ...
68060,81539e7eb97d9fba708603ac8f00b6c9,relogios_presentes,Péssima,Não comprem desta loja! Tentei fazer o cancela...,1,Péssima Não comprem desta loja! Tentei fazer ...
68364,bccbe20275c5ca847160e9ca42aa220d,relogios_presentes,Não entregou produto,"Sem comentário , não recebi o relógio Cassio .",1,"Não entregou produto Sem comentário , não rece..."


In [88]:
#LDA
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer

In [90]:
new_df.head()

,review_score,review_comment_message,clean_reviews
3,5.0,Recebi bem antes do prazo estipulado.,recebi bem antes prazo estipulado
4,5.0,Parabéns lojas lannister adorei comprar pela I...,parabéns lojas lannister adorei comprar intern...
9,4.0,aparelho eficiente. no site a marca do aparelh...,aparelho eficiente site marca aparelho impress...
12,4.0,"Mas um pouco ,travando...pelo valor ta Boa.\r\n",pouco travandopelo valor ta boa
15,5.0,"Vendedor confiável, produto ok e entrega antes...",vendedor confiável produto ok entrega antes prazo


In [92]:
# get the topics associated with the reviews
vectorizer = TfidfVectorizer().fit(new_df['clean_reviews'])
data_vectorized = vectorizer.transform(new_df['clean_reviews'])
lda_model = LatentDirichletAllocation(n_components=5).fit(data_vectorized)

In [94]:
def print_topics(model, vectorizer):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names_out()[i], topic[i])
                        for i in topic.argsort()[:-10 - 1:-1]])
        

print_topics(lda_model, vectorizer)

Topic 0:
[('recebi', 680.5911294694636), ('produto', 590.7482332488775), ('ainda', 454.1410384971967), ('comprei', 381.75988636079893), ('nao', 282.0154345482429), ('veio', 273.9811301365864), ('entregue', 244.83972736695702), ('pedido', 214.5738393436616), ('agora', 194.2447784022938), ('compra', 181.729840548202)]
Topic 1:
[('produto', 165.5650864769527), ('demorou', 133.4682435924915), ('otima', 123.16378264105505), ('rapido', 115.33218815230533), ('pouco', 108.09520984967214), ('veio', 97.14409688240693), ('ser', 91.81004953487631), ('achei', 84.92560344487462), ('pra', 81.10186807148939), ('ainda', 75.64145475786165)]
Topic 2:
[('certo', 323.0601072471267), ('produto', 292.0752594470425), ('gostei', 285.6414741326942), ('tudo', 243.85305930386875), ('boa', 219.459258996367), ('otimo', 214.8625330279081), ('entrega', 206.64083015940565), ('conforme', 185.55178494171767), ('combinado', 151.06616145097144), ('qualidade', 142.91544469213972)]
Topic 3:
[('prazo', 1495.8967978959554), (